# Bank salary - minorities

In [15]:
# importing libraries
import pandas as pd
from sklearn.model_selection import train_test_split


## Problem statement

Your client an American bank Scrooge McDuck Inc. based in Duckburg wants to know, if there exists a discrimination against minorities through different salaries. He provides you with a dataset containing next to the salaries of its employees information about their start salaries, years of education, gender, and an indicator of being part of a minority. As an expirienced data scientist you clearly know that OLS might help you here to understand the dependencies of the salary on different variables. Find out, if there might be a difference in salary depending on an employee's gender and/or minority affiliation. What role does the job category play?

Data Description
- SALARY: Yearly salary in US-dollars
- LOGSAL: Logarithmised salary
- EDUC: Education in years
- SALBEGIN: Starting salary
- LOGSALBEGIN: Logarithmised starting salary
- GENDER: Gender of the employee (0: female, 1: male)
- MINORITY: Minority affiliation (0: non minority, 1: minority)
- JOBCAT: Job category, with levels (1: admin, 2: custodial and 3: manage).

Interesting information missing:
- how long is an employee already working in this company.

## Plan

- importing data
- quick look at the data
- train test set
- data cleaning:
    - duplicates
    - missing values
    - rename columns
- EDA:
    - Hypothesis 1: Woman have lower salary than men
    - H 2: Minorities have lower salary than non minorities
    - H 3: More men and non minorities have JOBCAT 3
    - H 4: Higher educations will end up in higher JOBCAT
    - H 5: Minorities will have less salary raise than non minorities.
    
    - make some plots to see relationships between target and features (correlation plots / scatter plots)
    - maybe need some JOBCAT buckets and have different gender etc. information to be more specific about these problems
- Feature engineering:
    - dummy form of GENDER, MINORITY and JOBCAT (for modeling)
    - percentage of salary raise (H 5)
- Predictive modeling:
    - target/ y: SALARY
    - guess what's most effecting SALARY: EDUC, JOBCAT, GENDER, MINORITY
    - PROBLEM: there might be some effects from GENDER/MINORITIES influencing EDUC... be careful about causality!
    - start with simple model (binary regression (== linear regression with one feature) with highest correlating features) ... going for multiple linreg
    - METRIC: - profiling: adj. R2
              - predicting: RMSE
- Visualization
    - analysing the errors of our model (residual plots)
    - presenting results (maybe in a presentation)
    

In [5]:
# import data
bank_salary = pd.read_csv('data/bank_salary_data.asc', delimiter='\t')

In [7]:
# quick look at data
bank_salary.tail()

,IDNUMBER,SALARY,LOGSAL,EDUC,SALBEGIN,LOGSALBEGIN,GENDER,MINORITY,JOBCAT
469,470,26250,10.175421,12,15750,9.664596,1,1,1
470,471,26400,10.181119,15,15750,9.664596,1,1,1
471,472,39150,10.575156,15,15750,9.664596,1,0,1
472,473,21450,9.973480,12,12750,9.453287,0,0,1
473,474,29400,10.288750,12,14250,9.564512,0,0,1


In [8]:
bank_salary.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 474 entries, 0 to 473
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   IDNUMBER     474 non-null    int64  
 1   SALARY       474 non-null    int64  
 2   LOGSAL       474 non-null    float64
 3   EDUC         474 non-null    int64  
 4   SALBEGIN     474 non-null    int64  
 5   LOGSALBEGIN  474 non-null    float64
 6   GENDER       474 non-null    int64  
 7   MINORITY     474 non-null    int64  
 8   JOBCAT       474 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 33.5 KB


Result: no missing values!

In [9]:
bank_salary.describe()

,IDNUMBER,SALARY,LOGSAL,EDUC,SALBEGIN,LOGSALBEGIN,GENDER,MINORITY,JOBCAT
count,474.000000,474.000000,474.000000,474.000000,474.000000,474.000000,474.000000,474.000000,474.000000
mean,237.500000,34419.567511,10.356793,13.491561,17016.086498,9.669405,0.544304,0.219409,1.411392
std,136.976275,17075.661465,0.397334,2.884846,7870.638154,0.352841,0.498559,0.414284,0.773201
min,1.000000,15750.000000,9.664596,8.000000,9000.000000,9.104980,0.000000,0.000000,1.000000
25%,119.250000,24000.000000,10.085809,12.000000,12487.500000,9.432470,0.000000,0.000000,1.000000
50%,237.500000,28875.000000,10.270728,12.000000,15000.000000,9.615805,1.000000,0.000000,1.000000
75%,355.750000,36937.500000,10.516969,15.000000,17490.000000,9.769385,1.000000,0.000000,1.000000
max,474.000000,135000.000000,11.813030,21.000000,79980.000000,11.289532,1.000000,1.000000,3.000000


In [16]:
bank_salary.columns

Index(['IDNUMBER', 'SALARY', 'LOGSAL', 'EDUC', 'SALBEGIN', 'LOGSALBEGIN',
       'GENDER', 'MINORITY', 'JOBCAT'],
      dtype='object')

# Train test split

In [17]:
X = bank_salary[['IDNUMBER', 'EDUC', 'SALBEGIN', 'LOGSALBEGIN',
       'GENDER', 'MINORITY', 'JOBCAT']]
y = bank_salary[['SALARY']]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [20]:
df_train = pd.concat([X_train, y_train], axis=1)

In [24]:
df_train["MINORITY"].unique()

array([0, 1])

# Data cleaning

We already saw there are no missing values, and no unreasonable (in general statistics overview (.describe())).
Column names are reasonable, we won't change them.

We wanted to check for duplicates:

In [29]:
# checking if no ID is there more than once
bank_salary["IDNUMBER"].nunique() == bank_salary.shape[0]

True